# Data Processing

In [1]:
import os
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm
from pykrx import stock
from scipy.stats import zscore

In [2]:
folder_path = "data"
file_name = "DATA.xlsx"
file_path = os.path.join(folder_path, file_name)

In [3]:
sheet_list = pd.ExcelFile(file_path).sheet_names
list_dfs = []
for sheet in tqdm(sheet_list):
    temp = pd.read_excel(file_path,sheet_name=sheet,header=8,
                                  index_col=0, skiprows=range(9,14))
    temp = temp.astype("float64")
    temp.index.name = "Date"
    list_dfs.append(temp)
    temp = pd.DataFrame()

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:38<00:00,  2.77s/it]


In [4]:
print(sheet_list)

['CAP', 'GP', 'A', 'BPS', 'P', 'ADV20', 'RET20', 'VOL20', 'AVG20', 'HIGH52', 'LOW52', 'BETA1M3Y', 'KS200', 'ADJ_P']


In [5]:
df_cap, df_gp, df_a, df_bps, df_p, df_adv, df_ret = list_dfs[0:7]
df_vol, df_avg, df_high, df_low, df_beta, df_ks200 = list_dfs[7:13]
df_adjp = list_dfs[-1]

In [6]:
df_ks200.drop(df_ks200.columns[-1], axis=1, inplace=True)
df_ks200.rename(columns={df_ks200.columns[-1]:"KS200"}, inplace=True)

In [7]:
# scaling to unit
df_cap = df_cap * 1e6
df_gp = df_gp * 1e3
df_a = df_a * 1e3
df_ret = df_ret * 1e-2
df_ks200 = df_ks200 * 1e-2
df_vol = df_vol * 252 ** 0.5

In [8]:
# ratios
accounting_lag = 12
df_gp = df_gp.shift(accounting_lag)
df_a = df_a.shift(accounting_lag)
df_bps = df_bps.shift(accounting_lag)
df_gpa = (df_gp / df_a).fillna(method="ffill")
df_pbr = df_p / df_bps.fillna(method="ffill")

In [9]:
# capm alpha
df_alpha = df_ret - df_beta.mul(df_ks200["KS200"], axis=0)

In [22]:
# more simple features
df_voldiff = df_vol.diff(20)
df_advcap = df_adv / df_cap
df_zscore = (df_p - df_avg) / df_vol
df_hlpct = (df_p - df_low) / (df_high - df_low)

In [11]:
# rolling returns
df_ret1m = df_ret
df_ret3m = (df_ret+1).apply(np.log).rolling(3).sum().apply(np.exp)-1
df_ret6m = (df_ret+1).apply(np.log).rolling(6).sum().apply(np.exp)-1
df_ret1y = (df_ret+1).apply(np.log).rolling(12).sum().apply(np.exp)-1

In [12]:
# ar(1) beta coefficient t stat trend
def t_val(r2, n=20):
    return ((n - 2) * r2 / (1 - r2)) ** 0.5
df_t_d = df_adjp.rolling(20).corr(df_adjp.shift(1)).pow(2).apply(t_val)
df_t_d[df_t_d.index.name] = df_t_d.index
df_trend = df_t_d.resample('M').last()
df_trend.set_index(df_trend.columns[-1], drop=True, inplace=True)

In [13]:
pdf_dict = dict()
for i in tqdm(df_alpha.index):
    date_str = i.strftime("%Y%m%d")
    # KRX Web Data
    if i >= datetime.datetime(2014, 5, 1):
        temp = stock.get_index_portfolio_deposit_file("1028", date_str)
        temp = ["A"+x for x in temp]
        pdf_dict[i] = temp
    else:
        pass

100%|████████████████████████████████████████████████████████████████████████████████| 241/241 [00:48<00:00,  5.02it/s]


In [14]:
in_dict = dict()
for ticker, item in tqdm(df_alpha.iteritems()):
    temp_dict = dict()
    for dt in item.index:
        if dt not in pdf_dict.keys():
            temp_dict[dt] = np.nan
        else:
            if ticker in pdf_dict[dt]:
                temp_dict[dt] = 1.0
            else:
                temp_dict[dt] = np.nan
    in_dict[ticker] = temp_dict

df_pdf = pd.DataFrame(in_dict)

526it [00:00, 967.72it/s] 


In [23]:
# copy
df_alpha_c = df_alpha.copy()
df_pbr_c = df_pbr.copy()
df_gpa_c = df_gpa.copy()
df_voldiff_c = df_voldiff.copy()
df_advcap_c = df_advcap.copy()
df_zscore_c = df_zscore.copy()
df_hlpct_c = df_hlpct.copy()
df_ret1m_c = df_ret1m.copy()
df_ret3m_c = df_ret3m.copy()
df_ret6m_c = df_ret6m.copy()
df_ret1y_c = df_ret1y.copy()
df_trend_c = df_trend.copy()

In [24]:
# nan-out when not in index constituents
df_alpha_c = df_pdf * df_alpha_c
df_pbr_c = df_pdf * df_pbr_c
df_gpa_c = df_pdf * df_gpa_c
df_voldiff_c = df_pdf * df_voldiff_c
df_advcap_c = df_pdf * df_advcap_c
df_zscore_c = df_pdf * df_zscore_c
df_hlpct_c = df_pdf * df_hlpct_c
df_ret1m_c = df_pdf * df_ret1m_c
df_ret3m_c = df_pdf * df_ret3m_c
df_ret6m_c = df_pdf * df_ret6m_c
df_ret1y_c = df_pdf * df_ret1y_c
df_trend_c = df_pdf * df_trend_c

In [25]:
# drop out data unavailable period
df_alpha_c = df_alpha_c.loc[list(pdf_dict.keys())[0]:]
df_pbr_c = df_pbr_c.loc[list(pdf_dict.keys())[0]:]
df_gpa_c = df_gpa_c.loc[list(pdf_dict.keys())[0]:]
df_voldiff_c = df_voldiff_c.loc[list(pdf_dict.keys())[0]:]
df_advcap_c = df_advcap_c.loc[list(pdf_dict.keys())[0]:]
df_zscore_c = df_zscore_c.loc[list(pdf_dict.keys())[0]:]
df_hlpct_c = df_hlpct_c.loc[list(pdf_dict.keys())[0]:]
df_ret1m_c = df_ret1m_c.loc[list(pdf_dict.keys())[0]:]
df_ret3m_c = df_ret3m_c.loc[list(pdf_dict.keys())[0]:]
df_ret6m_c = df_ret6m_c.loc[list(pdf_dict.keys())[0]:]
df_ret1y_c = df_ret1y_c.loc[list(pdf_dict.keys())[0]:]
df_trend_c = df_trend_c.loc[list(pdf_dict.keys())[0]:]

In [26]:
# winsorizing
def winsorize_row(row):
    lower_limit = np.percentile(row, 1)
    upper_limit = np.percentile(row, 99)
    row[row < lower_limit] = lower_limit
    row[row > upper_limit] = upper_limit
    return row

df_alpha_c = df_alpha_c.apply(winsorize_row, axis=1)
df_pbr_c = df_pbr_c.apply(winsorize_row, axis=1)
df_gpa_c = df_gpa_c.apply(winsorize_row, axis=1)
df_voldiff_c = df_voldiff_c.apply(winsorize_row, axis=1)
df_advcap_c = df_advcap_c.apply(winsorize_row, axis=1)
df_zscore_c = df_zscore_c.apply(winsorize_row, axis=1)
df_hlpct_c = df_hlpct_c.apply(winsorize_row, axis=1)
df_ret1m_c = df_ret1m_c.apply(winsorize_row, axis=1)
df_ret3m_c = df_ret3m_c.apply(winsorize_row, axis=1)
df_ret6m_c = df_ret6m_c.apply(winsorize_row, axis=1)
df_ret1y_c = df_ret1y_c.apply(winsorize_row, axis=1)
df_trend_c = df_trend_c.apply(winsorize_row, axis=1)

In [27]:
# zscoring
def z_val(x):
    return zscore(x, nan_policy='omit')
# do not zscore df_alpha_c as it is y variable
df_pbr_c = df_pbr_c.apply(z_val, axis=1)
df_gpa_c = df_gpa_c.apply(z_val, axis=1)
df_voldiff_c = df_voldiff_c.apply(z_val, axis=1)
df_advcap_c = df_advcap_c.apply(z_val, axis=1)
df_zscore_c = df_zscore_c.apply(z_val, axis=1)
df_hlpct_c = df_hlpct_c.apply(z_val, axis=1)
df_ret1m_c = df_ret1m_c.apply(z_val, axis=1)
df_ret3m_c = df_ret3m_c.apply(z_val, axis=1)
df_ret6m_c = df_ret6m_c.apply(z_val, axis=1)
df_ret1y_c = df_ret1y_c.apply(z_val, axis=1)
df_trend_c = df_trend_c.apply(z_val, axis=1)

C:\Users\jimmy\anaconda3\lib\site-packages\numpy\core\_methods.py:230: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


In [28]:
stack_list = []
feature_list = ["PBR", "GPA", "VOLDIF", "ADVCAP", "ZSCORE", "HLPCT",
                "RET1M", "RET3M", "RET6M", "RET1Y", "TREND", "ALPHA"]
df_temp = pd.DataFrame()
for ticker in tqdm(df_alpha_c.columns):
    df_temp = pd.concat({feature_list[0]:df_pbr_c[ticker],
                        feature_list[1]:df_gpa_c[ticker],
                        feature_list[2]:df_voldiff_c[ticker],
                        feature_list[3]:df_advcap_c[ticker],
                        feature_list[4]:df_zscore_c[ticker],
                        feature_list[5]:df_hlpct_c[ticker],
                        feature_list[6]:df_ret1m_c[ticker],
                        feature_list[7]:df_ret3m_c[ticker],
                        feature_list[8]:df_ret6m_c[ticker],
                        feature_list[9]:df_ret1y_c[ticker],
                        feature_list[10]:df_trend_c[ticker],
                        feature_list[11]:df_alpha_c[ticker].shift(1)},
                        axis=1)
    df_temp = df_temp.iloc[1:]
    df_temp.reset_index(inplace=True)
    df_temp.rename(columns={"index":"Date"}, inplace=True)
    df_temp['Ticker'] = ticker
    df_temp.set_index(["Date", "Ticker"], inplace=True)
    stack_list.append(df_temp)
    df_temp = pd.DataFrame()
df_stack = pd.DataFrame()
df_stack = pd.concat(stack_list, axis=0)
df_stack = df_stack[~df_stack[feature_list[:-1]].isna().all(axis=1)]
df_stack = df_stack[~df_stack[feature_list[-1]].isna()]
df_stack.sort_values(['Date'], inplace=True)
# do not drop all nan since some data will not be available
df_stack

100%|███████████████████████████████████████████████████████████████████████████████| 526/526 [00:02<00:00, 190.27it/s]


PBR       GPA    VOLDIF    ADVCAP    ZSCORE  \
Date       Ticker                                                      
2014-06-30 A005930 -0.058008  1.021696  0.376161 -0.562885 -5.291161   
           A069960 -0.167693  0.131890 -0.229215 -0.301009  0.284992   
           A001230 -0.373015 -0.770166  0.823208  0.744649 -0.220384   
           A005850 -0.146763 -0.515661  0.161951  0.033038 -0.111254   
           A001440  0.064057 -0.736534 -2.930122 -0.363790 -0.186373   
...                      ...       ...       ...       ...       ...   
2023-08-31 A284740 -0.317314  2.009884  0.220499 -0.086501  0.119636   
           A005420  1.384424 -0.404601  0.151795  1.604598 -0.174884   
           A139480 -0.447076  0.345683  0.609825  0.605670 -0.229983   
           A006280 -0.233237  0.212315 -0.000882 -0.414615 -0.281337   
           A066570 -0.277360  0.819467 -1.275734 -0.216757 -0.497862   

                       HLPCT     RET1M     RET3M     RET6M     RET1Y  \
Date       Ticker                                                      
2014-06-30 A005930 -0.435005 -1.185438 -0.133927 -0.427470 -0.556327   
           A069960 -0.669247  0.432339 -0.227122 -0.683549 -0.504534   
           A001230 -1.498697 -2.116634 -2.455957 -1.789610 -1.289418   
           A005850  1.418236  1.623514  0.985774  1.341102  0.787576   
           A001440 -1.037843 -0.770654 -0.548468 -0.890713 -1.005384   
...                      ...       ...       ...       ...       ...   
2023-08-31 A284740 -0.888353  0.381427 -0.478345 -0.854848 -0.350195   
           A005420 -0.339069 -1.919916 -0.502443 -0.021920  0.921830   
           A139480 -1.272917 -0.286430 -0.596337 -1.097547 -0.358065   
           A006280 -0.775064  0.271678 -0.414998 -0.276165 -0.401835   
           A066570 -0.075022 -0.849642 -0.943933 -0.462644  0.033995   

                       TREND     ALPHA  
Date       Ticker                       
2014-06-30 A005930  1.575869  0.054439  
           A069960  0.171569 -0.018893  
           A001230  2.181459 -0.104789  
           A005850  1.123846  0.084937  
           A001440 -0.702883 -0.045352  
...                      ...       ...  
2023-08-31 A284740 -0.453240 -0.098690  
           A005420  0.655783  0.158709  
           A139480  0.882332 -0.030113  
           A006280 -0.675745 -0.058596  
           A066570 -0.064628 -0.179284  

[21602 rows x 12 columns]

In [29]:
stack_name = os.path.join(folder_path, "data_stack.h5")
df_stack.to_hdf(stack_name, key='df', mode='w')
# also save adjusted price for simulation
adjp_name = os.path.join(folder_path, "data_adjp.h5")
df_adjp.to_hdf(adjp_name, key='df', mode='w')